In [1]:
%matplotlib notebook
import math
import matplotlib.pyplot as plt
import numpy as np
import quaternion
import pandas
import sys
import os
from scipy.ndimage.filters import gaussian_filter1d

nano_to_sec = 1e09

# configure the size of plot
fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 10
fig_size[1] = 10

ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [ ]:
def plot_trajectory_with_global_north(position, global_norths, interval=100, length=1.0, c='b'):
    """
    This function draws global norths direction on the trajectory, along with the median global
    north direction.
    """
    assert global_norths.shape[0] == position.shape[0]
    plt.figure('global_north')
    plt.plot(position[:, 0], position[:, 1], color='r')
    for i in range(0, position.shape[0], interval):
        direction = global_norths[i]
        vtx = np.array([position[i], position[i] + direction * length])
        plt.plot(vtx[:, 0], vtx[:, 1], color=c)
    # Compute and plot the median yaw
    median_yaw = np.array([math.atan2(n[1], n[0]) for n in global_norths])
    median_yaw = np.median(median_yaw)
    median_dir = np.array([math.cos(median_yaw), math.sin(median_yaw)])
    median_dir = median_dir / np.linalg.norm(median_dir)
    pos_center = np.average(position[:, :2], axis=0)
    vtx = np.array([pos_center, pos_center + median_dir * length * 3.0])
    plt.plot(vtx[:, 0], vtx[:, 1], color='r', linewidth=3.0)
    plt.scatter(pos_center[0], pos_center[1], c='red', s=64)
    plt.axis('equal')

In [ ]:
# Replace it with your path
data_path = '../../../data_pocket/hang_leg_pos1'
data_all = pandas.read_csv(data_path + '/processed/data.csv')
positions = data_all[['pos_x', 'pos_y', 'pos_z']].values
orientations = data_all[['ori_w', 'ori_x', 'ori_y', 'ori_z']].values
gravity = data_all[['grav_x', 'grav_y', 'grav_z']].values
magnet = data_all[['magnet_x', 'magnet_y', 'magnet_z']].values

global_norths = np.empty(magnet.shape)
for i in range(global_norths.shape[0]):
    rotor = quaternion.quaternion(*orientations[i])
    global_norths[i] = (rotor * quaternion.quaternion(1.0, *magnet[i]) * rotor.conj()).vec

global_norths[:, 2] = 0
global_norths = global_norths / np.linalg.norm(global_norths, axis=1)[:, np.newaxis]

plot_trajectory_with_global_north(positions, global_norths, length=2.0)